# Scrap My Prop

### Laboratórios de Engenharia Informática

**"Development of an IT solution for the extraction and automatic analysis of data and relevant information for the calculation of land and properties."**

## Importar Bibliotecas Python

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.core.debugger import set_trace

# 'Imovirtual'

## Ler Dados do CSV

In [ ]:
data_imo = pd.read_csv('dados/dados_imovirtual_16_04.csv', engine='python', encoding='utf8')
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

## Pré-processamento comum a todo o Dataset

**Funções auxiliares**

In [ ]:
def ajustPreco(string):
    res = str(string)
    res = res.replace(" ", "")
    res = float(pd.to_numeric(res, errors='ignore'))
    return res

**Colunas 'Distrito' e 'Concelho': Retirar colunas de forma total porque já era uma pré-condição do projeto, mas primeiro verificar se, de facto, todos os campos estão preenchidos com 'Braga';**

In [ ]:
print("Distritos existentes:", data_imo['Distrito'].unique())
print("Concelhos existentes:", data_imo['Concelho'].unique())

In [ ]:
if 'Distrito' in data_imo.columns:
    data_imo = data_imo.drop(['Distrito'], axis = 1) 
if 'Concelho' in data_imo.columns:
    data_imo = data_imo.drop(['Concelho'], axis = 1) 

**Colunas 'Nome' e 'Id': Não são características dos imóveis mas sim identificadores, portanto não devem entrar nos cálculos (Drop);**

In [ ]:
if 'Nome' in data_imo.columns:
    data_imo = data_imo.drop(['Nome'], axis = 1) 
if 'Id' in data_imo.columns:
    data_imo = data_imo.drop(['Id'], axis = 1) 

**Dar join a algumas colunas que apesar de aparentarem ter nomes diferentes representam o mesmo**

**Também pus os valores de box 1 carro e box 2 carros em garagem box, mas não as eliminei, bem como estacionamento, parqueamento 1 carro e parqueamento 2 carros/ hidromassagem e jacuzzi/ adaptado(a) a mobilidade reduzida**

**Fazer o mesmo com piscina e piscina privada???**

In [ ]:
#pd.set_option('display.max_columns', None)
#data_imo.columns.to_numpy()

In [ ]:
for index,val in data_imo['Vista de cidade.1'].items():
    if (val==True):
        data_imo['Vista de cidade'].iloc[index] = True
for index,val in data_imo['Sótão'].items():
    if (val==True):
        data_imo['Sotão'].iloc[index] = True
for index,val in data_imo['Vista de Serra'].items():
    if (val==True):
        data_imo['Vista de campo/serra'].iloc[index] = True
for index,val in data_imo['Adaptada a mobilidade reduzida'].items():
    if (val==True):
        data_imo['Adaptado a mobilidade reduzida'].iloc[index] = True
for index,val in data_imo['Box 1 carro'].items():
    if (val==True):
        data_imo['Garagem box'].iloc[index] = True
for index,val in data_imo['Box 2 carros'].items():
    if (val==True):
        data_imo['Garagem box'].iloc[index] = True
for index,val in data_imo['Parqueamento (1 carro)'].items():
    if (val==True):
        data_imo['Estacionamento'].iloc[index] = True
for index,val in data_imo['Parqueamento (2 carros)'].items():
    if (val==True):
        data_imo['Estacionamento'].iloc[index] = True
for index,val in data_imo['Hidromassagem'].items():
    if (val==True):
        data_imo['Hidromassagem/jacuzzi'].iloc[index] = True
for index,val in data_imo['Jacuzzi'].items():
    if (val==True):
        data_imo['Hidromassagem/jacuzzi'].iloc[index] = True
        
data_imo.drop(['Vista de cidade.1', 'Sótão', 'Vista de Serra', 'Hidromassagem', 'Jacuzzi', 'Adaptada a mobilidade reduzida'], axis=1)


In [ ]:
#data_imo.info(verbose=True, null_counts=True)

**Normalizar os valores de todas as áreas**

In [ ]:
for column in data_imo.columns:
    if "Área" in column and "Área administrativa" not in column and "Área florestal" not in column:
        for i in range(len(data_imo[column])):
            if not pd.isnull(data_imo[column].iloc[i]):
                data_imo[column].iloc[i]=str(data_imo[column].iloc[i])[0:-3]

**Coluna 'Preço': Drop dos imóveis com valor 'nan'; Transformar em dado numérico;**

In [ ]:
data_imo['Preço'] = data_imo['Preço'].apply(ajustPreco)

nan_prices = data_imo['Preço'].index[data_imo['Preço'].apply(np.isnan)]
data_imo = data_imo.drop(nan_prices)
data_imo.index = np.arange(1, len(data_imo) + 1)

data_imo['Preço'] = data_imo['Preço'].apply(int)

In [ ]:
data_imo.info(verbose=True)

**Coluna 'Freguesia': Não podem existir valores nulos nesta coluna, senão estamos a perder o principal fator de diferenciação dos preços;**

In [ ]:
data_imo['Freguesia'].isnull().values.any()

**Ver os valores únicos de cada coluna para proceder ao processamento**

In [ ]:
'''
for col in data_imo.columns:
    uniques=data_imo[col].unique()
    print("Valores únicos para a coluna ", col, ": ", uniques)
'''

**Dividir os dados para a tipologia moradias**

In [ ]:
data_imo_moradias = data_imo.loc[(data_imo['Tipo de imóvel'] == 'Moradia')]
data_imo_moradias.index = np.arange(1, len(data_imo_moradias) + 1)

# Dados - Moradias

In [ ]:
data_imo_moradias.head()

In [ ]:
data_imo_moradias.info(verbose=True, null_counts=True)

## Análise exploratória de dados

*Distribuição da feature 'Preço'*

In [ ]:
fig = plt.figure(figsize = (20,5))
sns.set_style('darkgrid')
data_imo_moradias['Preço'].hist(bins=30)
plt.xlabel('Preço')

In [ ]:
fig = plt.figure(figsize = (10,3))
sns.boxplot(x=data_imo_moradias['Preço'])

*Relação entre o Preço e outras variáveis*

In [ ]:
fig = plt.figure(figsize = (30,15))

ax1 = fig.add_subplot(2,3,1)
ax1.scatter(data_imo_moradias['Área útil m/2'], data_imo_moradias['Preço'])
ax1.set_xlabel('Área útil')
ax1.set_ylabel('Preço')

*Relacionamento do 'Preço' com outras features categóricas importantes*

In [ ]:
cat_data = data_imo_moradias[['Freguesia']]

In [ ]:
for cat in cat_data.dtypes[:10].index.values:
    plt.figure(figsize=(20, 8))
    plt.xticks(rotation=90)
    sns.boxplot(x=cat, y='Preço', data=data_imo_moradias) 
    sns.swarmplot(x=cat, y='Preço', data=data_imo_moradias)
    plt.show()

**Remoção de outliers (Quantos eliminar???)**

In [ ]:
outliers = data_imo_moradias.loc[data_imo_moradias['Preço'] > 1000000].index
data_imo_moradias = data_imo_moradias.drop(outliers)
data_imo_moradias.index = np.arange(1, len(data_imo_moradias) + 1)
data_imo_moradias.shape

In [ ]:
fig = plt.figure(figsize = (20,5))
sns.set_style('darkgrid')
data_imo_moradias['Preço'].hist(bins=30)
plt.xlabel('Preço')

**Dados em falta por coluna**

In [ ]:
pd.set_option('display.max_rows', None)
percent_missing = data_imo_moradias.isnull().sum() * 100 / len(data_imo_moradias)
missing_value_data_imo_moradias_columns = pd.DataFrame({'percent_missing (%)': percent_missing})
sort_data = missing_value_data_imo_moradias_columns.copy()
sort_data.sort_values('percent_missing (%)', inplace=True, ascending=False)
sort_data

**Remover colunas que tenham mais de 30% (?) de missing values**

In [ ]:
a_manter = list(missing_value_data_imo_moradias_columns.index[missing_value_data_imo_moradias_columns['percent_missing (%)'] < 30])
data_imo_moradias = data_imo_moradias[a_manter]
data_imo_moradias